In [1]:
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.SparkContext._
import org.apache.spark.rdd.RDD
import org.apache.spark.rdd.PairRDDFunctions._
import org.apache.spark.SparkContext
import org.apache.spark.graphx.Edge
import org.apache.spark.graphx.TripletFields
import org.apache.spark.graphx.Graph
import org.apache.spark.graphx.VertexId
import org.apache.spark.graphx.EdgeTriplet
import org.apache.spark.graphx.GraphLoader
import org.apache.spark._
import org.apache.spark.graphx._
import scala.util.Random
import scala.io.Source
import java.io._

Intitializing Scala interpreter ...

Spark Web UI available at http://a95775a1cae0:4040
SparkContext available as 'sc' (version = 2.3.1, master = local[*], app id = local-1618514211651)
SparkSession available as 'spark'


import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.SparkContext._
import org.apache.spark.rdd.RDD
import org.apache.spark.rdd.PairRDDFunctions._
import org.apache.spark.SparkContext
import org.apache.spark.graphx.Edge
import org.apache.spark.graphx.TripletFields
import org.apache.spark.graphx.Graph
import org.apache.spark.graphx.VertexId
import org.apache.spark.graphx.EdgeTriplet
import org.apache.spark.graphx.GraphLoader
import org.apache.spark._
import org.apache.spark.graphx._
import scala.util.Random
import scala.io.Source
import java.io._


In [2]:
// Class to store vertex special attributes
class VertexAttr(
    var degree : Int = 0,
    var initialLabel : Boolean = false,
    var vertexLabels : Array[Float] = Array[Float](0, 0, 0)
) extends java.io.Serializable {     
    
    def setDegree(degree: Int) = {
        this.degree = degree
    }
}

defined class VertexAttr


In [3]:
// Special function to read label file and transform it into a Map object
def readLabel(filename: String) = {
    val pairs = 
        for {
            line <- Source.fromFile(filename).getLines()
            split = line.split(" ").map(_.trim)
            name = split.head.toInt
            scores = split.tail(0).toInt
        } yield (name -> scores)
        pairs.toMap
}

readLabel: (filename: String)scala.collection.immutable.Map[Int,Int]


In [4]:
// Load initial Graph
val initialGraph = GraphLoader.edgeListFile(sc, "./email-eu-core/email-Eu-core.txt", canonicalOrientation=false)
val initialLabel = readLabel("./email-eu-core/email-Eu-core-department-labels.txt")
val n_label = initialLabel.map({ case (k, v) => v }).max + 1

initialGraph: org.apache.spark.graphx.Graph[Int,Int] = org.apache.spark.graphx.impl.GraphImpl@8fb7298
initialLabel: scala.collection.immutable.Map[Int,Int] = Map(645 -> 1, 892 -> 38, 69 -> 15, 809 -> 22, 629 -> 1, 365 -> 14, 138 -> 21, 760 -> 17, 101 -> 16, 479 -> 1, 347 -> 0, 846 -> 1, 909 -> 4, 333 -> 10, 628 -> 7, 249 -> 14, 893 -> 14, 518 -> 35, 962 -> 9, 468 -> 1, 234 -> 19, 941 -> 41, 0 -> 1, 777 -> 9, 555 -> 14, 666 -> 14, 88 -> 20, 481 -> 22, 352 -> 0, 408 -> 21, 977 -> 19, 170 -> 13, 523 -> 4, 582 -> 15, 762 -> 21, 115 -> 13, 683 -> 22, 730 -> 23, 217 -> 37, 276 -> 4, 994 -> 21, 308 -> 1, 741 -> 7, 5 -> 25, 873 -> 15, 449 -> 1, 120 -> 1, 247 -> 14, 379 -> 7, 878 -> 11, 440 -> 4, 655 -> 4, 511 -> 0, 614 -> 24, 269 -> 17, 677 -> 26, 202 -> 4, 597 -> 23, 861 -> 5, 10 -> 9, 385 -> ...

In [5]:
// Setup 
val rand = new Random(42)
val probability = 0.5

def random_label(random: Random): Int = {
    random.nextInt(3)
}

// Transform function to apply to all vertices of the graph
def transform(VD: VertexId): VertexAttr = {
    val label = initialLabel(VD.toInt)
    var labelArray = Array.ofDim[Float](n_label)
    var va = new VertexAttr(0, false, labelArray);
    
    if (rand.nextFloat() < probability){
        labelArray(label) = 1
        va.initialLabel = true
        va.vertexLabels = labelArray
    }
    
    va;
}

rand: scala.util.Random = scala.util.Random@272fd9fa
probability: Double = 0.5
random_label: (random: scala.util.Random)Int
transform: (VD: org.apache.spark.graphx.VertexId)VertexAttr


In [6]:
// Transform vertices into VertexAttr with a random label assigned
val graph = initialGraph.mapVertices({ case (id, attr) => transform(id)})

graph: org.apache.spark.graphx.Graph[VertexAttr,Int] = org.apache.spark.graphx.impl.GraphImpl@6a04a43d


In [7]:
// Populate Graph degree for each vertices and create final graph
val vertex_rdd = graph.degrees.zip(graph.vertices).map({ case (vDeg, vAttr) => vAttr._2.setDegree(vDeg._2) ; vAttr})
val edge_rdd = graph.edges
val graph2 = Graph(vertex_rdd, edge_rdd)

// Edges go both sides ways
val finalGraph = Graph(graph2.vertices, graph2.edges.union(graph2.edges.reverse))

vertex_rdd: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, VertexAttr)] = MapPartitionsRDD[20] at map at <console>:66
edge_rdd: org.apache.spark.graphx.EdgeRDD[Int] = EdgeRDDImpl[14] at RDD at EdgeRDD.scala:41
graph2: org.apache.spark.graphx.Graph[VertexAttr,Int] = org.apache.spark.graphx.impl.GraphImpl@3e90a153
finalGraph: org.apache.spark.graphx.Graph[VertexAttr,Int] = org.apache.spark.graphx.impl.GraphImpl@8928038


In [8]:
// Vertex Progam
def VertexProgram(id : VertexId, 
                  VD : VertexAttr, 
                  A : Array[Float]) : VertexAttr  = {
    var newVertexAttr = new VertexAttr()
    newVertexAttr.degree = VD.degree
    newVertexAttr.initialLabel = VD.initialLabel
    newVertexAttr.vertexLabels = VD.vertexLabels
    
    // If the node does not have a label
    if(VD.initialLabel == false){
        newVertexAttr.vertexLabels = A.map(_ / VD.degree)
    }
    newVertexAttr
}

// Send Message
def SendMsg(triplet : EdgeTriplet[VertexAttr, Int]): Iterator[(VertexId, Array[Float])] = {
    Iterator((triplet.dstId, triplet.srcAttr.vertexLabels))
}

// Merge Message
def MergeMsg(A : Array[Float],
             B : Array[Float]) : Array[Float] = {
    val mergeMsg = new Array[Float](A.length);
    for ( i <- 0 to (A.length - 1)) {
        mergeMsg(i) = A(i) + B(i);
    }
    mergeMsg;
}

VertexProgram: (id: org.apache.spark.graphx.VertexId, VD: VertexAttr, A: Array[Float])VertexAttr
SendMsg: (triplet: org.apache.spark.graphx.EdgeTriplet[VertexAttr,Int])Iterator[(org.apache.spark.graphx.VertexId, Array[Float])]
MergeMsg: (A: Array[Float], B: Array[Float])Array[Float]


In [9]:
// finalGraph.vertices.foreach(v => println(v._1, v._2.degree, v._2.initialLabel, v._2.vertexLabels.mkString(" ")))

In [15]:
val initialMsg = Array.ofDim[Float](n_label)
val maxIter = 20   // Stop after maxIter iterations

val graphLP = finalGraph.pregel(initialMsg, maxIter, EdgeDirection.Either)(
    (id, VD, newLabelsArray) => VertexProgram(id, VD, newLabelsArray),
    tr => SendMsg(tr),
    (a, b) => MergeMsg(a, b)
)

2021-04-15 19:25:02 WARN  BlockManager:66 - Block rdd_1798_0 already exists on this machine; not re-adding it


initialMsg: Array[Float] = Array(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
maxIter: Int = 2
graphLP: org.apache.spark.graphx.Graph[VertexAttr,Int] = org.apache.spark.graphx.impl.GraphImpl@7d73ce


In [16]:
graphLP.vertices.foreach(v => println(v._1, v._2.initialLabel, v._2.vertexLabels.mkString(",")))

(451,false,0.0010869566,0.0016304348,0.0,0.0,0.0029146536,0.0016545893,5.434783E-4,0.0,0.0,3.7037037E-4,5.434783E-4,0.0,0.0,0.0033091786,0.6089061,0.004656085,0.0027657007,0.0,0.0,0.0033091786,5.434783E-4,5.434783E-4,0.0,0.0021980677,0.0,0.0039157984,7.4074074E-4,0.0,0.0,0.0,0.0,0.0,0.006031746,0.0,0.0010869566,5.434783E-4,0.0062869736,0.007941799,5.434783E-4,0.0,0.0,0.0)
(893,true,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(454,true,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(386,false,0.1482023,0.055263378,0.0,0.0,0.012297066,0.0028538813,0.0017123288,0.24735065,0.003164557,0.0015822785,0.006381863,0.0054953382,7.48503E-4,7.48503E-4,0.0130715035,0.0033870614,0.0,0.0014534884,0.0,9.3984965E-4,0.0,0.0,0.0029069767,0.003031608,0.0,0.002906

(204,true,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(514,false,8.8183425E-4,0.0,0.0,0.0,0.005228017,0.0,0.0,0.0,0.43020302,0.004067816,0.0039235526,5.6689343E-4,5.6689343E-4,0.0014487276,0.0020156212,0.17251645,8.8183425E-4,8.8183425E-4,0.0,4.4091712E-4,0.0014487276,8.8183425E-4,0.0038788512,8.8183425E-4,0.0,4.4091712E-4,0.0870862,5.6689343E-4,0.0,0.0014487276,0.0,0.0,0.08219955,0.0,0.0043197684,0.0022675737,0.014938353,0.0034196475,4.4091712E-4,0.0,0.0015747041,0.0)
(390,false,0.049791407,0.024086067,7.424732E-5,0.0011516623,0.030004688,0.0041012326,0.0055625364,0.42846048,5.168461E-4,8.314202E-4,0.007394421,0.005144194,6.7876425E-4,0.009681748,0.011058796,0.0027722244,7.8838924E-4,0.002885269,0.0,0.0011887944,6.5050233E-4,0.0011977861,7.856402E-4,0.0034455094,0.0,0.003187593,0.017757257,6.90645E-4,1.4849464E-4,0.0024490352,0.0,4.787911E-4,0.002893087,0.0,0.00

(781,true,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(21,false,0.01052222,0.038498838,3.2192527E-4,0.009612632,0.016480424,0.0077564614,0.035997033,0.0070363227,0.011757063,0.08717792,0.034619078,0.02152034,0.0012154133,0.012374357,0.027986294,0.086624034,0.01140441,0.022030016,0.0,0.008378163,0.011790416,0.014401414,0.001982015,0.020656442,3.0525032E-5,0.019435486,0.04194069,0.0016751153,9.677414E-4,0.010224524,1.8939395E-4,0.009744492,0.011183811,0.0,0.016651584,0.01258092,0.06628762,0.013841462,0.024615971,4.54557E-4,0.009623102,2.0525451E-5)
(577,true,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(367,true,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

(731,true,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(453,false,0.0067140507,0.017227516,3.6208716E-4,0.0013603145,0.042110007,0.0041895667,0.0068310057,0.005105143,0.0017566462,0.0053065456,0.012761891,0.005597262,0.0035008555,0.055244073,0.020931084,0.03643893,0.0049050497,0.0037303283,0.0,0.009214304,0.0032051282,0.0029663676,0.0014261607,0.0076663624,1.5926104E-4,0.0025791796,0.04775324,0.0041730735,0.0011833173,0.0010932331,0.0,0.0032292511,0.0033714876,0.0,0.051116914,0.027522579,0.2497226,0.031483214,0.0049623935,0.0011947406,0.02377557,0.0)
(787,false,0.004166667,0.025,0.0,0.008333334,0.0,0.004166667,0.033333335,0.004166667,0.008333334,0.06666667,0.025,0.016666668,0.0,0.004166667,0.0125,0.06666667,0.008333334,0.016666668,0.0,0.004166667,0.008333334,0.008333334,0.0,0.016666668,0.0,0.016666668,0.033333335,0.0,0.0,0.008333334,0.0,0.008333334,0.008333334,0.0

(646,true,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(808,false,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(758,true,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0)
(842,true,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(528,false,0.0383432,0.0019236209,0.0,0.001566478,0.011159831,0.0021994342,7.142857E-4,0.01824611,0.0013472419,0.0020665447,0.005278322,7.142857E-4,8.547009E-4,0.005770863,0.0049186703,0.0034087696,0.004485215,0.0064729387,0.0,0.0072710626,0.0049852817,0.073822364,0.009034532,0.0026

...

(693,true,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0)
(109,false,0.0016937669,0.022954402,3.387534E-4,0.0065713604,0.017361285,0.25342417,0.0021926584,0.0037078098,0.25190905,0.009823545,0.021395985,0.007864631,6.775068E-4,0.01190458,0.022535732,0.013149336,0.010389429,0.008590652,0.0,0.005850588,0.0038180677,0.0028084219,0.0028084219,0.012803968,0.0,3.387534E-4,0.0,0.0053398334,0.0034241853,0.0012932704,0.0,0.004046563,0.008308393,0.0,0.0031405608,0.0012932704,0.037889287,0.0046623265,0.0034859285,0.0,0.0021926584,0.002130915)
(167,true,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(519,false,0.005627561,0.21773721,0.0,0.0010910132,0.0121608665,0.0037179086,0.011514866,0.014608455,0.0010871798,0.09978314,0.0047185873,0.0016883903,1.5926104E-

(805,false,0.0,0.0056818184,0.0,0.0,0.0085227275,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.44363636,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0)
(457,true,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(615,false,0.0073563852,0.018246157,0.0044444446,6.0059014E-4,0.055363804,0.013130455,0.0019890317,0.004421004,0.0048250463,0.0038032234,0.039991725,0.0139128715,0.024048772,0.028122457,0.04524325,0.03818901,0.0042171166,0.0070131817,0.0,0.061773915,0.002780607,0.007664848,0.0030934208,0.0143110715,1.4064698E-4,0.014720889,0.012227701,0.014308334,0.0012294342,0.0014431959,0.0,0.02389755,0.004219051,0.0,0.031088514,0.049186446,0.07169334,0.023049856,0.014849141,3.0430267E-4,0.0045437054,0.0)
(669,false,0.0025,0.005,0.0,0.0010714286,0.014603175,0.002936508,7.142857E-4,0.0,3.5714285E-4,3.5714285E-4,0.017738095

(169,false,0.0042896364,0.010416045,1.7864605E-4,0.0027406511,0.07973619,0.014303636,0.012358144,0.003199232,0.013319467,0.015489334,0.08459064,0.04791758,0.011576059,0.15685253,0.021928247,0.0076376055,0.0143259335,0.013417174,0.0,0.016030544,0.024564447,0.011686246,0.0027886736,0.013984232,3.6447796E-5,0.013249488,0.019928342,0.0010977405,8.662374E-4,0.011123747,0.0,7.2124973E-4,0.0027765445,0.0,0.026251446,0.002233325,0.06192823,0.017922873,0.01978774,2.6138333E-4,0.0062720957,6.0304537E-5)
(261,true,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(395,false,0.043941446,0.004107838,1.7834849E-4,0.0035578327,0.050055448,0.010520036,0.001136935,0.32770374,3.618445E-4,0.0017529918,0.0038756274,0.001229663,4.148399E-4,0.0026106855,0.0068524317,0.0024458957,0.0019369565,0.034973953,0.0,0.0013481494,5.783317E-4,9.280914E-4,9.928605E-4,0.0013478915,5.8142916E-5,0.0339700

(455,false,6.421046E-4,0.0014064136,0.0,6.6391943E-4,0.031333532,0.0013063683,0.0015381649,3.3655597E-4,0.021630164,0.0020627277,0.30910012,0.025359573,0.023010941,0.25318554,0.0034676488,0.0013944332,0.001302294,0.0026993568,0.0,0.004485408,0.026963277,0.0051656887,0.0056379167,0.0010248397,0.0,0.023053994,0.0046824263,1.9034886E-4,0.0012702557,6.816161E-4,0.0,1.3736264E-4,0.002145024,0.0,0.001703481,8.136689E-4,0.00889342,0.0024994714,0.001267553,0.0,2.5701078E-4,0.0)
(475,false,0.0,0.009090909,0.0,0.0,0.009090909,0.0,0.0,0.0,0.5090909,0.0045454544,0.013636364,0.0,0.0,0.0,0.013636364,0.0,0.03181818,0.018181818,0.0,0.013636364,0.009090909,0.0,0.0045454544,0.009090909,0.0,0.013636364,0.009090909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009090909,0.0,0.03181818,0.0,0.027272727,0.0,0.0,0.0)
(619,true,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
(325,false,0.0017904831,0.005054

(365,false,0.0340976,0.048260346,2.9456816E-5,0.0021012237,0.06315359,0.0147639075,0.014037785,0.0041776965,0.0027862971,0.007663253,0.025556657,0.0056265676,5.7436444E-4,0.034908503,0.14914255,0.011414491,0.016820177,0.0060315947,0.0,0.027101463,0.014870776,0.014152079,0.0019397065,0.025909921,0.0,0.015132124,0.008392465,0.0013868291,0.0010611248,0.0017370656,0.0,0.0011562771,0.0028315887,0.0,0.031753235,0.014609469,0.06994202,0.003951484,0.016286014,2.900196E-4,0.0015967191,1.2351779E-4)
(155,false,0.049979012,0.028895263,0.0,0.001216931,0.023922501,0.005988368,0.0021887063,0.029697936,0.018303173,0.023188401,0.040609613,0.019131424,0.017865257,0.021944687,0.06078247,0.03835178,0.030465826,0.026458558,0.0,0.040614523,0.0031591787,0.029445628,0.021206059,0.037295315,0.0,0.0020305628,0.033901747,0.0103747845,0.0011810005,9.913949E-4,0.0,6.437208E-4,0.003635002,0.0,0.0075552175,0.040468942,0.022566592,0.0060969824,0.022405,6.36506E-4,0.0012728202,9.796239E-5)
(315,true,0.0,1.0,0.0,0.0,0